In [154]:
import numpy as np
from scipy import sparse
import xgboost as xgb
import pandas as pd
import re
import string
import time

from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import feature_selection
from itertools import product

import osgeo 
import fiona
import json
import geopandas as gpd
import geocoder
from scipy import sparse
from shapely.geometry import Point
from geopandas.tools import sjoin
from geopy.distance import vincenty

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

In [2]:
train_data = pd.read_json(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\train.json')

test_data = pd.read_json(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\test.json') 

In [3]:
test_data["bathrooms"].loc[19671] = 1.5 
test_data["bathrooms"].loc[22977] = 2.0 
test_data["bathrooms"].loc[63719] = 2.0

In [4]:
import random
index=list(range(train_data.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_data)
b=[np.nan]*len(train_data)
c=[np.nan]*len(train_data)

for i in range(5):
    building_level={}
    for j in train_data['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*train_data.shape[0])/5):int(((i+1)*train_data.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_data.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_data.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_data['manager_level_low']=a
train_data['manager_level_medium']=b
train_data['manager_level_high']=c



a=[]
b=[]
c=[]
building_level={}
for j in train_data['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(train_data.shape[0]):
    temp=train_data.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_data['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_data['manager_level_low']=a
test_data['manager_level_medium']=b
test_data['manager_level_high']=c

manager_variable = ['manager_level_low','manager_level_medium','manager_level_high']

In [5]:
train_size = train_data.shape[0]

### Create target variables

In [6]:
train_data['target'] = train_data['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
train_data['low'] = train_data['interest_level'].apply(lambda x: 1 if x=='low' else 0)
train_data['medium'] = train_data['interest_level'].apply(lambda x: 1 if x=='medium' else 0)
train_data['high'] = train_data['interest_level'].apply(lambda x: 1 if x=='high' else 0)

## Merge training and testing data

In [7]:
full_data=pd.concat([train_data
                       ,test_data])

In [8]:
from collections import defaultdict, Counter
features = full_data[["features"]].apply(
    lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])
n = 5

feature_counts = Counter()
for feature in features.features:
    feature_counts.update(feature)
feature = sorted([k for (k,v) in feature_counts.items() if v > n])
feature[:10]

def clean(s):
    x = s.replace("-", "")
    x = x.replace(" ", "")
    x = x.replace("twenty four hour", "24")
    x = x.replace("24/7", "24")
    x = x.replace("24hr", "24")
    x = x.replace("24-hour", "24")
    x = x.replace("24hour", "24")
    x = x.replace("24 hour", "24")
    x = x.replace("common", "cm")
    x = x.replace("concierge", "doorman")
    x = x.replace("bicycle", "bike")
    x = x.replace("private", "pv")
    x = x.replace("deco", "dc")
    x = x.replace("decorative", "dc")
    x = x.replace("onsite", "os")
    x = x.replace("outdoor", "od")
    x = x.replace("ss appliances", "stainless")
    return x

def feature_hash(x):
    cleaned = clean(x, uniq)
    key = cleaned[:4].strip()
    return key

key2original = defaultdict(list)
k = 4
for f in feature:
    cleaned = clean(f)
    key = cleaned[:k].strip()
    key2original[key].append(f)

def to_tuples():
    for f in feature:
        key = clean(f)[:k].strip()
        yield (f, key2original[key][0])
        
deduped = list(to_tuples())
#df = pd.DataFrame(deduped, columns=["original_feature", "unique_feature"])

In [9]:
 full_data["cfeatures"] = full_data[["features"]].replace(key2original)

In [10]:
full_data.ix[4620, 'price']  = 1025
full_data.ix[12168, 'price']  = 3400
full_data.ix[32611, 'price']  = 10000

full_data.ix[51229,'latitude'] = 40.831835
full_data.ix[51229,'longitude'] = -73.921021
full_data.ix[113035, 'latitude'] = 40.869699
full_data.ix[113035,'longitude'] = -73.243134
full_data.ix[104822, 'latitude'] = 40.752816
full_data.ix[104822,'longitude'] = -73.970854
full_data.ix[72896, 'latitude'] = 40.772661
full_data.ix[72896,'longitude'] = -73.955558
full_data.ix[78568, 'latitude'] = 40.763670
full_data.ix[78568,'longitude'] = -73.958609
full_data.ix[109135, 'latitude'] = 40.778075
full_data.ix[109135,'longitude'] = -73.952238
full_data.ix[17772, 'latitude'] = 40.748368
full_data.ix[17772,'longitude'] = -73.976599
full_data.ix[67902, 'latitude'] = 40.805668
full_data.ix[67902,'longitude'] = -73.941987
full_data.ix[21168, 'latitude'] = 40.731960
full_data.ix[21168,'longitude'] = -74.002091
full_data.ix[55585, 'latitude'] = 40.737547
full_data.ix[55585,'longitude'] = -73.984084
full_data.ix[45416, 'latitude'] = 40.754539
full_data.ix[45416,'longitude'] = -73.998372
full_data.ix[17772, 'latitude'] = 40.747632
full_data.ix[17772,'longitude'] = -73.974814
full_data.ix[109135, 'latitude'] = 40.775065
full_data.ix[109135,'longitude'] = -73.948073
full_data.ix[39798, 'latitude' ] = 40.747632
full_data.ix[39798,'longitude' ] = -73.974814

In [11]:
missing_data = pd.read_csv(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\a_few_missing2.csv')

In [12]:
full_data.fillna(0, inplace = True)
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'latitude'] = missing_data.lat.values
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'longitude'] = missing_data.long.values

# Geo Processing

missingCoords = full_data[(full_data.longitude == 0) | (full_data.latitude == 0)]
missingGeoms = (missingCoords.street_address + ', New York').apply(geocoder.google)

full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'latitude'] = missingGeoms.apply(lambda x: x.lat)
full_data.loc[(full_data.longitude == 0) | (full_data.latitude == 0), 'longitude'] = missingGeoms.apply(lambda x: x.lng)

missing_data = pd.DataFrame({'lat':missingGeoms.apply(lambda x: x.lat), 'long':missingGeoms.apply(lambda x: x.lng)})
missing_data.to_csv(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\a_few_missing2.csv')

In [13]:
full_data['geometry'] = full_data.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [14]:
poly = gpd.GeoDataFrame.from_file(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\nynta.geojson')
gdat = gpd.GeoDataFrame(full_data, crs = poly.crs, geometry='geometry')
geo_data = sjoin(gdat,poly, how='left', op='within')

In [15]:
pd.set_option('display.max_columns', 40)
geo_data.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,high,interest_level,latitude,listing_id,longitude,low,manager_id,manager_level_high,manager_level_low,manager_level_medium,medium,photos,price,street_address,target,cfeatures,geometry,index_right,borocode,boroname,cartodb_id,countyfips,created_at,ntacode,ntaname,shape_area,shape_leng,updated_at
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],0.0,medium,40.7145,7211212,-73.9425,0.0,5ba989232d0489da1b5f2c45f6688adc,0.000000,0.746479,0.253521,1.0,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,1.0,[],POINT (-73.9425 40.7145),29.0,3.0,Brooklyn,90.0,047,2015-07-16T02:50:22,BK90,East Williamsburg,3.914106e+07,49188.619678,2015-07-16T02:50:22
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",0.0,low,40.7947,7150865,-73.9667,1.0,7533621a882f71e25173b27e3139d83d,0.000000,1.000000,0.000000,0.0,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,0.0,"[Doorman, Elevator, Fitness Center, Cats Allow...",POINT (-73.9667 40.7947),147.0,1.0,Manhattan,124.0,061,2015-07-16T02:50:22,MN12,Upper West Side,3.438105e+07,29158.132670,2015-07-16T02:50:22
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",1.0,high,40.7388,6887163,-74.0018,0.0,d9039c43983f6e564b1482b273bd7b01,0.045045,0.603604,0.351351,0.0,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,2.0,"[Laundry In Building, Dishwasher, Hardwood Flo...",POINT (-74.0018 40.7388),86.0,1.0,Manhattan,46.0,061,2015-07-16T02:50:22,MN23,West Village,2.500053e+07,29385.030305,2015-07-16T02:50:22
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",0.0,low,40.7539,6888711,-73.9677,1.0,1067e078446a7897d2da493d2f741316,0.046358,0.827815,0.125828,0.0,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,0.0,"[Hardwood Floors, No Fee]",POINT (-73.96769999999999 40.7539),134.0,1.0,Manhattan,109.0,061,2015-07-16T02:50:22,MN19,Turtle Bay-East Midtown,1.739787e+07,21638.781206,2015-07-16T02:50:22
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],0.0,low,40.8241,6934781,-73.9493,1.0,98e13ad4b495b9613cef886d79a6291f,0.000000,1.000000,0.000000,0.0,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,0.0,[Pre-War],POINT (-73.94929999999999 40.8241),133.0,1.0,Manhattan,151.0,061,2015-07-16T02:50:22,MN04,Hamilton Heights,1.609379e+07,17410.823299,2015-07-16T02:50:22


In [16]:
geo_var = ['boroname','borocode','ntaname']
full_data = pd.merge(left = full_data, right = geo_data[geo_var],left_index=True,right_index=True, how = 'left')

### Group Variables

In [17]:
num_vars = ['bathrooms','bedrooms','latitude','longitude','price']
cat_vars = ['building_id','manager_id','display_address','street_address', 'borocode','ntanew']
text_vars = ['description','features']
date_var = 'created'
image_var = 'photos'


In [18]:
full_data['ntanew'] = full_data[['boroname', 'ntaname']].apply(lambda x: x[1] if (x[0] == 1) or (x[0] ==3) else x[0], axis = 1)

### Date variable

In [19]:
full_data['created_datetime'] = pd.to_datetime(full_data['created'], format="%Y-%m-%d %H:%M:%S")

In [184]:
# full_data['created_year']=full_data['created_datetime'].apply(lambda x:x.year) ## low variant
full_data['created_month']=full_data['created_datetime'].apply(lambda x:x.month)
full_data['created_day']=full_data['created_datetime'].apply(lambda x:x.day)
full_data['created_dayofweek']=full_data['created_datetime'].apply(lambda x:x.dayofweek)
full_data['created_dayofyear']=full_data['created_datetime'].apply(lambda x:x.dayofyear)
full_data['created_weekofyear']=full_data['created_datetime'].apply(lambda x:x.weekofyear)
full_data['created_hour']=full_data['created_datetime'].apply(lambda x:x.hour)
full_data['created_epoch']=full_data['created_datetime'].apply(lambda x:x.value//10**9)



In [218]:
date_num_vars = ['created_month','created_dayofweek','created_dayofyear'
                 ,'created_weekofyear','created_hour','created_epoch', 'photo_created_hour', 'adv2']#, 'photo_created_dayofyear']

In [166]:
full_data['photo_created'] =  pd.to_datetime(full_data['time_stamp'],unit='s')

In [214]:
full_data['photo_created'][50:]

50       2016-11-09 18:09:22
51       2016-11-09 18:09:40
52       2016-11-09 18:10:02
53       2016-11-09 18:10:08
54       2016-11-09 18:10:10
55       2016-11-09 18:10:46
56       2016-11-09 18:10:52
57       2016-11-22 02:54:46
58       2016-11-09 18:11:00
59       2016-11-02 23:36:42
60       2016-11-09 18:11:12
61       2016-11-09 18:11:18
62       2016-11-09 18:11:34
63       2016-11-09 18:11:56
64       2016-11-09 18:12:04
65       2016-11-22 02:54:54
66       2016-11-09 18:12:40
67       2016-11-22 02:54:54
68       2016-11-09 18:13:18
69       2016-11-09 18:13:32
70       2016-11-09 18:13:48
71       2016-11-09 18:14:02
72       2016-11-09 18:14:12
73       2016-11-22 02:54:54
74       2016-11-09 18:14:24
75       2016-11-09 18:14:40
76       2016-11-09 18:15:00
77       2016-11-09 18:15:06
78       2016-11-09 18:15:16
79       2016-11-09 18:15:40
                 ...        
123981   2016-11-09 17:57:06
123982   2016-11-09 17:57:10
123983   2016-11-09 17:57:14
123984   2016-

In [215]:
full_data['photo_created_hour']=full_data['photo_created'].apply(lambda x:x.hour)
full_data['photo_created_dayofyear']=full_data['photo_created'].apply(lambda x:x.dayofyear)
#full_data['photo_created_hour']=full_data['photo_created'].apply(lambda x:x.hour)
full_data['adv1']=full_data['time_stamp']/full_data['created_epoch']
full_data['adv2']=full_data['photo_created_hour']/full_data['created_hour']

## Additional Numeric Variables

In [21]:
# full_data['price']=full_data['price'].apply(np.log)

In [22]:
full_data['rooms'] = full_data['bedrooms'] + full_data['bathrooms'] 
full_data['num_of_photos'] = full_data['photos'].apply(lambda x:len(x))
full_data['num_of_features'] = full_data['features'].apply(lambda x:len(x))
full_data['len_of_desc'] = full_data['description'].apply(lambda x:len(x))
full_data['words_of_desc'] = full_data['description'].apply(lambda x:len(re.sub('['+string.punctuation+']', '', x).split()))


full_data['nums_of_desc'] = full_data['description']\
        .apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: len([s for s in x if s.isdigit()]))
        
full_data['has_phone'] = full_data['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)
full_data['has_email'] = full_data['description'].apply(lambda x: 1 if '@renthop.com' in x else 0)
full_data['price_latitue'] = (full_data["price"])/ (full_data["latitude"]+1.0) 
full_data['price_longtitude'] =  (full_data["price"])/ (full_data["longitude"]-1.0)  

additional_num_vars = ['rooms','num_of_photos','num_of_features','len_of_desc',
                    'words_of_desc','has_phone','has_email','price_latitue', 'price_longtitude']

### Numeric interactions

In [23]:
full_data['avg_word_len'] = full_data[['len_of_desc','words_of_desc']]\
                                    .apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
    
full_data['price_per_room'] = full_data[['price','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bedroom'] = full_data[['price','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_bathroom'] = full_data[['price','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
full_data['price_per_photo'] = full_data[['price','num_of_photos']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)


full_data['photos_per_room'] = full_data[['num_of_photos','rooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)

interactive_num_vars = ['avg_word_len','price_per_room','price_per_bedroom','price_per_bathroom','price_per_photo',
                        'photos_per_room']

### Categorical Features

#### 1. Label Encoding

In [24]:
full_data.borocode.fillna(6, inplace = True)
full_data.ntanew.fillna('other', inplace = True)

In [25]:
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
for cat_var in cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var])
    LE_vars.append(LE_var)
    LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (LE_vars))

Label Encoding building_id
Label Encoding manager_id
Label Encoding display_address
Label Encoding street_address
Label Encoding borocode
Label Encoding ntanew
Label-encoded feaures: ['building_id_le', 'manager_id_le', 'display_address_le', 'street_address_le', 'borocode_le', 'ntanew_le']


In [26]:
LE_vars.remove('ntanew_le')

#### 2. One Hot Encoding

In [27]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_sparse=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))


OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

One-hot-encoding finished in 0.335424 seconds
OHE_sparse size : (124011, 57874)
One-hot encoded catgorical feature samples : ['building_0', 'building_00005cb939f9986300d987652c933e15', 'building_00024d77a43f0606f926e2312513845c', 'building_000ae4b7db298401cdae2b0ba1ea8146', 'building_0012f1955391bca600ec301035b97b65', 'building_0021440c04241281a436ec21accc40b1', 'building_002d1eba40aa0a6610e04ff20543585f', 'building_003d8740e21484dcc2280639b25539a4', 'building_00480e54b53fe77d17964be3f8307a99', 'building_00553d95d22484bcc36831c9248d1dbc', 'building_0055c8662ba19e95f78df97592d2b83e', 'building_0056dbdf2881b76f2a0171eb753ec9e0', 'building_0059ae562b9e338a59eaf962cb3eedd2', 'building_005e0f8d7fb7b92be351cbf1dd985149', 'building_0067f166111490e7af7f1a878a67bb5e', 'building_0070bc94a3f80aa717bb15708e98ba54', 'building_0071cda335745940cdae1dc31abfc701', 'building_0078281cd69f4bfec17e42e5cf5eecd9', 'building_0078c2ab46afba9969637ac83621901e', 'building_007ae1cd90420f18bad7b6892a9a1411', 'buil

#### 3. Leave-one-out Encoding

Based on the paper "A Preprocessing Scheme for High-Cardinality Categorical Attributes in Classification and Prediction Problems"

http://helios.mm.di.uoa.gr/~rouvas/ssi/sigkdd/sigkdd.vol3.1/barreca.ps

** A couple of Kaggle scripts: **

R version: by Braden Murray: https://www.kaggle.com/brandenkmurray/two-sigma-connect-rental-listing-inquiries/it-is-lit/comments

Python Version 1, by Stanislav Ushakov
https://www.kaggle.com/stanislavushakov/two-sigma-connect-rental-listing-inquiries/python-version-of-it-is-lit-by-branden/comments

Python Version 2, by Rakhlin
https://www.kaggle.com/rakhlin/two-sigma-connect-rental-listing-inquiries/another-python-version-of-it-is-lit-by-branden/code


In [28]:
##Create a function to encode high-cardinality cateogrical features

def designate_single_observations(df1, df2, column):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    return df1, df2


def hcc_encode(train_df, test_df, variable, target, prior_prob, k, f=1, g=1, r_k=None, update_df=None):
    """
    See "A Preprocessing Scheme for High-Cardinality Categorical Attributes in
    Classification and Prediction Problems" by Daniele Micci-Barreca
    """
    hcc_name = "_".join(["hcc", variable, target])

    grouped = train_df.groupby(variable)[target].agg({"size": "size", "mean": "mean"})
    grouped["lambda"] = 1 / (g + np.exp((k - grouped["size"]) / f))
    grouped[hcc_name] = grouped["lambda"] * grouped["mean"] + (1 - grouped["lambda"]) * prior_prob

    df = test_df[[variable]].join(grouped, on=variable, how="left")[hcc_name].fillna(prior_prob)
    if r_k: df *= np.random.uniform(1 - r_k, 1 + r_k, len(test_df))     # Add uniform noise. Not mentioned in original paper

    if update_df is None: update_df = test_df
    if hcc_name not in update_df.columns: update_df[hcc_name] = np.nan
    update_df.update(df)
    return

In [29]:
for col in ('building_id', 'manager_id', 'display_address'):
    train_data, test_data = designate_single_observations(train_data, test_data, col)
    
prior_low, prior_medium, prior_high = train_data[["low", "medium", "high"]].mean() 

skf = model_selection.StratifiedKFold(5)
attributes = product(("building_id", "manager_id"), zip(("medium", "high"), (prior_medium, prior_high)))
for variable, (target, prior) in attributes:
    hcc_encode(train_data, test_data, variable, target, prior, k=5, r_k=None)
    for train, test in skf.split(np.zeros(len(train_data)), train_data['interest_level']):
        hcc_encode(train_data.iloc[train], train_data.iloc[test], variable, target, prior, k=5, r_k=0.01,
                   update_df=train_data)
        
hcc_data = pd.concat([train_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']],
           test_data[['building_id', 'manager_id', 'display_address',
            'hcc_building_id_medium','hcc_building_id_high',
            'hcc_manager_id_medium','hcc_manager_id_high']]
           ]
          )
full_data['building_id'] = hcc_data['building_id']
full_data['manager_id'] = hcc_data['manager_id']
full_data['display_address'] = hcc_data['display_address']
full_data['hcc_building_id_medium'] = hcc_data['hcc_building_id_medium']
full_data['hcc_building_id_high'] = hcc_data['hcc_building_id_high']
full_data['hcc_manager_id_medium'] = hcc_data['hcc_manager_id_medium']
full_data['hcc_manager_id_high'] = hcc_data['hcc_manager_id_high']
hcc_vars = ['hcc_building_id_medium','hcc_building_id_high','hcc_manager_id_medium','hcc_manager_id_high']    

# Clustering

In [30]:
full_data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'high', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'low', 'manager_id', 'manager_level_high',
       'manager_level_low', 'manager_level_medium', 'medium', 'photos',
       'price', 'street_address', 'target', 'cfeatures', 'geometry',
       'boroname', 'borocode', 'ntaname', 'ntanew', 'created_datetime',
       'created_month', 'created_day', 'created_dayofweek',
       'created_dayofyear', 'created_weekofyear', 'created_hour',
       'created_epoch', 'rooms', 'num_of_photos', 'num_of_features',
       'len_of_desc', 'words_of_desc', 'nums_of_desc', 'has_phone',
       'has_email', 'price_latitue', 'price_longtitude', 'avg_word_len',
       'price_per_room', 'price_per_bedroom', 'price_per_bathroom',
       'price_per_photo', 'photos_per_room', 'building_id_le', 'manager_id_le',
       'display_address_le', 'street_address_le', 'borocode_le', 'ntanew_le',

In [31]:
from sklearn.cluster import DBSCAN
scale = preprocessing.StandardScaler()
cluster_vars = ['created_weekofyear','latitude', 'longitude', 'bedrooms']
cluster_df = full_data[cluster_vars]
cluster_df['bedrooms'] = cluster_df['bedrooms'].clip_upper(5)
cluster_df['bedrooms'] = cluster_df['bedrooms'].map(lambda x: 0.7 * x)
cluster_df['created_weekofyear'] = cluster_df['created_weekofyear'].map(lambda x: 6 * x)
cluster_df[cluster_vars] = scale.fit_transform(cluster_df[cluster_vars])
db = DBSCAN(eps=0.28, min_samples=25).fit(cluster_df)

In [32]:
labels = db.labels_

In [33]:
label_set = set(labels)

In [34]:
len(label_set)

24

In [35]:
full_data['clusters'] = labels

In [36]:
geo = ['latitude', 'longitude']

In [37]:
geo = ['latitude', 'longitude']
full_data['coords'] = list(zip(full_data.latitude, full_data.longitude))

In [38]:
full_data['coords'].head(1)

10    (40.7145, -73.9425)
Name: coords, dtype: object

### Text Features

#### 1. Features

In [36]:
full_data["cfeatures"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
cntvec = CountVectorizer(stop_words='english', max_features=200)
feature_sparse =cntvec.fit_transform(full_data["features"]\
                                     .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x])))

feature_vars = ['cfeature_' + v for v in cntvec.vocabulary_]

#### 2. Description

##### tf-idf not working, instead, using CountVectorizer, not working either

In [37]:
# tfidf = TfidfVectorizer(stop_words='english', max_features=10)
# desc_sparse = tfidf.fit_transform(full_data["description"])
# desc_vars = ['desc_' + v for v in tfidf.get_feature_names()]

In [38]:
cntvec = CountVectorizer(stop_words='english', max_features=100)
desc_sparse = cntvec.fit_transform(full_data["description"])
desc_vars = ['desc_' + v for v in cntvec.vocabulary_]

##### word2vec - to be added

#### Street Address

In [39]:
# embedding

cntvec = CountVectorizer(stop_words='english', max_features=10)
st_addr_sparse = cntvec.fit_transform(full_data["street_address"])
st_addr_vars = ['desc_' + v for v in cntvec.vocabulary_]

In [40]:
full_data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'high', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'low', 'manager_id', 'manager_level_high',
       'manager_level_low', 'manager_level_medium', 'medium', 'photos',
       'price', 'street_address', 'target', 'cfeatures', 'geometry',
       'boroname', 'borocode', 'ntaname', 'ntanew', 'created_datetime',
       'created_month', 'created_day', 'created_dayofweek',
       'created_dayofyear', 'created_weekofyear', 'created_hour',
       'created_epoch', 'rooms', 'num_of_photos', 'num_of_features',
       'len_of_desc', 'words_of_desc', 'nums_of_desc', 'has_phone',
       'has_email', 'price_latitue', 'price_longtitude', 'avg_word_len',
       'price_per_room', 'price_per_bedroom', 'price_per_bathroom',
       'price_per_photo', 'photos_per_room', 'building_id_le', 'manager_id_le',
       'display_address_le', 'street_address_le', 'borocode_le', 'ntanew_le',

In [41]:
full_data['boro_bed'] = full_data['borocode'].map(str) + full_data[ 'bedrooms'].clip_upper(4).astype(str)

#### image vars - to be added

### Numberic vs Categorical Interactions

In [42]:
price_by_manager = full_data.groupby('manager_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_manager.columns = ['manager_id','min_price_by_manager',
                            'max_price_by_manager','median_price_by_manager','mean_price_by_manager']
full_data = pd.merge(full_data,price_by_manager, how='left',on='manager_id')

created_epoch_by_manager = full_data.groupby('manager_id')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
created_epoch_by_manager.columns = ['manager_id','min_created_epoch_by_manager',
                            'max_created_epoch_by_manager','median_created_epoch_by_manager','mean_created_epoch_by_manager']
full_data = pd.merge(full_data,created_epoch_by_manager, how='left',on='manager_id')


price_by_building = full_data.groupby('building_id')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_price_by_building',
                            'max_price_by_building','median_price_by_building','mean_price_by_building']
full_data = pd.merge(full_data,price_by_building, how='left',on='building_id')


created_epoch_by_building = full_data.groupby('building_id')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_created_epoch_by_building',
                            'max_created_epoch_by_building','median_created_epoch_by_building','mean_created_epoch_by_building']
full_data = pd.merge(full_data,price_by_building, how='left',on='building_id')

price_by_disp_addr = full_data.groupby('display_address')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_disp_addr.columns = ['display_address','min_price_by_disp_addr',
                            'max_price_by_disp_addr','median_price_by_disp_addr','mean_price_by_disp_addr']
full_data = pd.merge(full_data,price_by_disp_addr, how='left',on='display_address')




full_data['price_percentile_by_manager']=\
            full_data[['price','min_price_by_manager','max_price_by_manager']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_building']=\
            full_data[['price','min_price_by_building','max_price_by_building']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
full_data['price_percentile_by_disp_addr']=\
            full_data[['price','min_price_by_disp_addr','max_price_by_disp_addr']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)


full_data['created_epoch_percentile_by_manager']=\
            full_data[['created_epoch','min_created_epoch_by_manager','max_created_epoch_by_manager']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
        
price_by_borocode = full_data.groupby('borocode')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_borocode.columns = ['borocode','min_price_by_borocode',
                            'max_price_by_borocode','median_price_by_borocode','mean_price_by_borocode']
full_data = pd.merge(full_data,price_by_borocode, how='left',on='borocode')

full_data['price_percentile_by_borocode']=\
            full_data[['price','min_price_by_borocode','max_price_by_borocode']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)
        
price_by_boro_bed = full_data.groupby('boro_bed')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_boro_bed.columns = ['boro_bed','min_price_by_boro_bed',
                            'max_price_by_boro_bed','median_price_by_boro_bed','mean_price_by_boro_bed']
full_data = pd.merge(full_data,price_by_boro_bed, how='left',on='boro_bed')

full_data['price_percentile_by_boro_bed']=\
            full_data[['price','min_price_by_boro_bed','max_price_by_boro_bed']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)

In [43]:
price_by_cluster = full_data.groupby('clusters')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_cluster.columns = ['clusters','min_price_by_clusters',
                            'max_price_by_clusters','median_price_by_clusters','mean_price_by_clusters']
full_data = pd.merge(full_data,price_by_cluster, how='left',on='clusters')


In [44]:
price_by_ntanew = full_data.groupby('ntanew')['price'].agg([np.min,np.max,np.median,np.mean]).reset_index()
price_by_ntanew.columns = ['ntanew','min_price_by_ntanew',
                            'max_price_by_ntanew','median_price_by_ntanew','mean_price_by_ntanew']
full_data = pd.merge(full_data,price_by_ntanew, how='left',on='ntanew')

full_data['price_percentile_by_ntanew']=\
            full_data[['price','min_price_by_ntanew','max_price_by_ntanew']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)

In [45]:
full_data['price_percentile_by_clusters']=\
            full_data[['price','min_price_by_clusters','max_price_by_clusters']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)


In [46]:
created_epoch_by_display_address = full_data.groupby('display_address')['created_epoch'].agg([np.min,np.max,np.median,np.mean]).reset_index()
created_epoch_by_display_address.columns = ['display_address','min_created_epoch_by_display_address',
                            'max_created_epoch_by_display_address','median_created_epoch_by_display_address','mean_created_epoch_by_display_address']
full_data = pd.merge(full_data,created_epoch_by_display_address, how='left',on='display_address')


In [47]:
created_dayofyear_by_created_month = full_data.groupby('created_month')['created_dayofyear'].agg([np.min,np.max,np.median,np.mean]).reset_index()
created_dayofyear_by_created_month.columns = ['created_month','min_created_dayofyear_by_created_month',
                            'max_created_dayofyear_by_created_month','median_price_by_created_dayofyear_month','mean_created_dayofyear_by_created_month']
full_data = pd.merge(full_data,created_dayofyear_by_created_month, how='left',on='created_month')

full_data['created_dayofyear_percentile_by_created_month']=\
            full_data[['created_dayofyear','min_created_dayofyear_by_created_month','max_created_dayofyear_by_created_month']]\
            .apply(lambda x:(x[0]-x[1])/(x[2]-x[1]) if (x[2]-x[1])!=0 else 0.5,
                  axis=1)

In [48]:
num_cat_vars = ['median_price_by_manager','mean_price_by_manager',
                'median_price_by_building','mean_price_by_building',
                'median_price_by_disp_addr','mean_price_by_disp_addr',
                'median_created_epoch_by_manager','mean_created_epoch_by_manager',
                'price_percentile_by_manager','price_percentile_by_building',
                'price_percentile_by_disp_addr','created_epoch_percentile_by_manager',
                'median_price_by_clusters', 'mean_price_by_clusters', 
                'price_percentile_by_clusters',
                'median_created_epoch_by_display_address',#'mean_created_epoch_by_display_address',
                'median_price_by_boro_bed','mean_price_by_boro_bed',
                #'median_price_by_ntanew',
                'mean_price_by_ntanew','price_percentile_by_ntanew',
                'created_dayofyear_percentile_by_created_month'
               ]

### Listing ID matters

In [53]:
min_listing_id = full_data['listing_id'].min()
max_listing_id = full_data['listing_id'].max()
full_data['listing_id_pos']=full_data['listing_id'].apply(lambda x:np.float64((x-min_listing_id+1))/(max_listing_id-min_listing_id+1))

In [160]:
num_vars = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price', 'listing_id', 'listing_id_pos', 'time_stamp']

## Centroid

In [55]:
centroid = (full_data.latitude.sum(axis = 0)/len(full_data)\
    ,full_data.longitude.sum(axis = 0)/len(full_data))

In [56]:
from scipy.spatial import distance
geo = ['latitude', 'longitude']
full_data['coords'] = list(zip(full_data.latitude, full_data.longitude))
def dist_to_center(x):
    true_center = np.array((40.7128,-74.0059)).reshape(-1,2)
    k = distance.cdist(np.array(x).reshape(-1,2), true_center, 'euclidean') 
    return k.astype(float)
full_data['dist_to_center'] = full_data['coords'].map(dist_to_center)
full_data['dist_to_center'] = full_data['dist_to_center'].str[0]
full_data['dist_to_center'] = full_data['dist_to_center'].str[0]

In [200]:
dist_var = ['dist_to_center']#, 'simi']

In [81]:
full_data['pos'] = full_data.longitude.round(3).astype(str) + '_' + full_data.latitude.round(3).astype(str)
vals = full_data['pos'].value_counts()
dvals = vals.to_dict()
full_data['density'] = full_data['pos'].apply(lambda x: dvals.get(x, vals.max()))

In [84]:
len(set(full_vars)), len(full_vars)

(62, 62)

# Interaction

In [98]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
def interactions(dataframe):
    combos = list(combinations(list(dataframe.columns), 5))
    polynomial = PolynomialFeatures(interaction_only = False, include_bias = False)
    dataframe = polynomial.fit_transform(dataframe)
    dataframe = pd.DataFrame(dataframe)
    noint_indicies = [i for i, x in enumerate(list((dataframe == 0).all())) if x]
    dataframe = dataframe.drop(dataframe.columns[noint_indicies], axis = 1)
    return dataframe 

In [155]:
leak = pd.read_csv(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\listing_image_time.csv')

In [195]:
similar = pd.read_csv(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\s3.csv')

In [156]:
leak.head()

,Listing_Id,time_stamp
0,6811957,1479785186
1,6811958,1479786880
2,6811960,1479780964
3,6811964,1479783510
4,6811965,1479786168


In [209]:
similar.columns = ['dz', 'density']

In [198]:
similar.head()

,dz,simi
0,7170325,1.000000
1,7092344,0.692308
2,7158677,0.800000
3,7211212,0.826087
4,7225292,0.800000


In [159]:
full_data = pd.merge(full_data, leak, left_on = 'listing_id', right_on = 'Listing_Id', how = 'left')

In [199]:
full_data = pd.merge(full_data, similar, left_on = 'listing_id', right_on = 'dz', how = 'left')

In [99]:
intr = interactions(full_data[['price_per_bedroom', 'dist_to_center']])

In [91]:
full_data['int1'] = intr['price_per_bedroom_dist_to_center']

In [212]:
num_vars = ['bathrooms', 'bedrooms',  'price2','latitude', 'longitude', 'listing_id','listing_id_pos', 'time_stamp']# 'listing_id', #'latitude', 'longitude'

In [206]:
full_data['price2'] = np.log(full_data.price)

### Create training and testing data

In [219]:
%%time


##Baseline with features from "features" and street address

full_vars = num_vars + date_num_vars\
+ additional_num_vars + interactive_num_vars + LE_vars + hcc_vars + num_cat_vars + manager_variable + dist_var #+ intr_vars
#,st_addr_sparse 
train_x = sparse.hstack([full_data[full_vars],feature_sparse]).tocsr()[:train_size]
train_y = full_data['target'][:train_size].values

test_x = sparse.hstack([full_data[full_vars], feature_sparse]).tocsr()[train_size:]
test_y = full_data['target'][train_size:].values



print ("training data size: ", train_x.shape,"testing data size: ", test_x.shape)


params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.1
params['max_depth'] = 6
params['min_child_weight'] = 1
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['gamma'] = 1
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=5,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(50)])


training data size:  (49352, 268) testing data size:  (74659, 268)
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[232]	train-mlogloss:0.322181+0.00174294	test-mlogloss:0.517714+0.00589766

Wall time: 4min 16s


In [59]:
from sklearn.ensemble import ExtraTreesClassifier
fs = ExtraTreesClassifier(n_estimators = 50)
fs.fit(train_x, train_y)



ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [62]:
feature_importance = pd.DataFrame(sorted(zip(full_vars,fs.feature_importances_)
                          , key=lambda x: x[1], reverse = False),columns=['feature_name','importance']) 


In [82]:
d = feature_importance[220:].feature_name.tolist()

In [84]:
train_g = sparse.hstack([full_data[d],feature_sparse,st_addr_sparse]).tocsr()[:train_size]
train_z = full_data['target'][:train_size].values

test_g = sparse.hstack([full_data[d], feature_sparse,st_addr_sparse]).tocsr()[train_size:]
test_z = full_data['target'][train_size:].values


print ("training data size: ", train_g.shape,"testing data size: ", test_g.shape)


params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.1
params['max_depth'] = 6
params['min_child_weight'] = 1
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['gamma'] = 1
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_g, label=train_z.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=5,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(50)])

training data size:  (49352, 242) testing data size:  (74659, 242)
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[237]	train-mlogloss:0.375739+0.00157655	test-mlogloss:0.552332+0.00624041



In [87]:
%%time
clf = xgb.XGBClassifier(learning_rate = 0.1
                  , n_estimators =246
                  , max_depth = 6
                  , min_child_weight = 1
                  , subsample = 0.7
                  , colsample_bytree = 0.7
                  , gamma = 1
                  , seed = 1234
                  , nthread = -1
                  )




clf.fit(train_x, train_y)

feature_importance = pd.DataFrame(sorted(zip(full_vars,clf.feature_importances_)
                          , key=lambda x: x[1], reverse = True),columns=['feature_name','importance']) 

print (feature_importance.query('importance>0'))

                                     feature_name  importance
0                            hcc_building_id_high    0.032992
1                               price_per_bedroom    0.031587
2                          hcc_building_id_medium    0.031213
3                             hcc_manager_id_high    0.029496
4                   price_percentile_by_disp_addr    0.029403
5                                  dist_to_center    0.028653
6                                  price_per_room    0.027405
7                           hcc_manager_id_medium    0.026968
8                     price_percentile_by_manager    0.026531
9                                 price_per_photo    0.026156
10                              manager_level_low    0.026032
11            created_epoch_percentile_by_manager    0.024284
12                                   avg_word_len    0.024190
13                                 building_id_le    0.023784
14                                       latitude    0.023597
15      

In [53]:
feature_importance.to_csv(r'C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\f.csv')

## Manuanl Tuning

* Greedy-search
* Tune one parameter a time
* The results can be used for further tuning (by Bayesian Optimizer)

In [101]:
%%time
xgb_scores = pd.DataFrame()
scores = []
for max_depth in [3,4,5,6,7,8,9,10]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = max_depth
    params['min_child_weight'] = 1
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (max_depth,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_max_depth = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['max_depth'].values[0])
print ('best max_depth is', best_max_depth)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1005]	train-mlogloss:0.42341+0.00119897	test-mlogloss:0.534216+0.0067717

3 0.5342156 1006
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[561]	train-mlogloss:0.403971+0.00151469	test-mlogloss:0.533798+0.00632543

4 0.5337976 562
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[352]	train-mlogloss:0.385278+0.00205391	test-mlogloss:0.534787+0.00661267

5 0.5347868 353
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[257]	train-mlog

In [48]:
%%time
scores = []
for min_child_weight in [1,3,10,30,100]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = min_child_weight
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (min_child_weight,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_min_child_weight = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['min_child_weight'].values[0])
print ('best min_child_weight is', best_min_child_weight)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[567]	train-mlogloss:0.404098+0.000663067	test-mlogloss:0.533047+0.00679302

1 0.5330466 568
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[604]	train-mlogloss:0.398721+0.00242664	test-mlogloss:0.533035+0.00644301

3 0.5330354 605
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[578]	train-mlogloss:0.405064+0.00152694	test-mlogloss:0.532609+0.00630762

10 0.532609 579
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[591]	train-mlo

In [49]:
%%time
scores = []
for colsample_bytree in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = 1
    params['colsample_bytree'] = colsample_bytree
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (colsample_bytree,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_colsample_bytree = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['colsample_bytree'].values[0]


print ('best colsample_bytree is', best_colsample_bytree)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[711]	train-mlogloss:0.406754+0.00166083	test-mlogloss:0.531408+0.0071967

0.2 0.5314078 712
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[707]	train-mlogloss:0.399901+0.00127526	test-mlogloss:0.531204+0.00676155

0.3 0.531204 708
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[631]	train-mlogloss:0.406864+0.00186313	test-mlogloss:0.531466+0.00677792

0.4 0.5314664 632
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[613]	train-

In [50]:
%%time
scores = []
for subsample in [0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = subsample
    params['colsample_bytree'] = best_colsample_bytree
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (subsample,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_subsample = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['subsample'].values[0]

print ('best subsample is', best_subsample)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[630]	train-mlogloss:0.407024+0.00155329	test-mlogloss:0.533097+0.0076172

0.6 0.533097 631
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[610]	train-mlogloss:0.409901+0.00195984	test-mlogloss:0.531872+0.006592

0.7 0.5318718 611
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[734]	train-mlogloss:0.390355+0.00153484	test-mlogloss:0.530475+0.00693869

0.8 0.5304752 735
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[653]	train-ml

In [51]:
%%time
scores = []
for gamma in [0,0.5,1,1.5,2]:

    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['subsample'] = best_subsample
    params['colsample_bytree'] = best_colsample_bytree
    params['gamma'] = gamma
    params['seed']=1234

    cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
                   num_boost_round=1000000,
                   nfold=5,
           metrics={'mlogloss'},
           seed=1234,
           callbacks=[xgb.callback.early_stop(50)])
    best_iteration = len(cv_results)
    best_score = cv_results['test-mlogloss-mean'].min()
    print (gamma,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_gamma = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['gamma'].values[0]

print ('best gamma is', best_gamma)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[734]	train-mlogloss:0.390355+0.00153484	test-mlogloss:0.530475+0.00693869

0 0.5304752 735
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[727]	train-mlogloss:0.39159+0.00140822	test-mlogloss:0.531322+0.00675369

0.5 0.5313218 728
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[615]	train-mlogloss:0.409871+0.00171513	test-mlogloss:0.531511+0.00670895

1 0.5315106 616
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[746]	train-mlo

## Automated Tuning

* https://github.com/fmfn/BayesianOptimization


In [172]:
from bayes_opt import BayesianOptimization

xgtrain = xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1))

def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma):
    params = dict()
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eta'] = 0.1
    params['max_depth'] = int(max_depth )   
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = colsample_bytree
    params['subsample'] = subsample
    params['gamma'] = gamma
    params['verbose_eval'] = True    


    cv_result = xgb.cv(params, xgtrain,
                       num_boost_round=100000,
                       nfold=5,
                       metrics={'mlogloss'},
                       seed=1234,
                       callbacks=[xgb.callback.early_stop(50)])

    return -cv_result['test-mlogloss-mean'].min()


xgb_BO = BayesianOptimization(xgb_evaluate, 
                             {'max_depth': (3, 9),
                              'min_child_weight': (0, 100),
                              'colsample_bytree': (0.1, 0.8),
                              'subsample': (0.7, 1),
                              'gamma': (0, 2)
                             }
                            )

xgb_BO.maximize(init_points=6, n_iter=120)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[777]	train-mlogloss:0.395416+0.00063035	test-mlogloss:0.515122+0.00658389

    1 | 05m44s |   -0.51512 |             0.3588 |    1.2582 |      4.1649 |            80.6301 |      0.9264 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[163]	train-mlogloss:0.312694+0.00204997	test-mlogloss:0.516923+0.00569277

    2 | 03m48s |   -0.51692 |             0.6347 |    1.0328 |      8.0868 |            14.6203 |      0.7682 | 
Multiple eval metrics have b

In [173]:
## Show tuning results
BO_scores = pd.DataFrame(xgb_BO.res['all']['params'])
BO_scores['score'] = pd.DataFrame(xgb_BO.res['all']['values'])
BO_scores = BO_scores.sort_values(by='score',ascending=False)

BO_scores.head(8)

,colsample_bytree,gamma,max_depth,min_child_weight,subsample,score
50,0.190996,1.984392,7.564569,45.368728,0.996902,-0.513230
61,0.170692,1.917625,6.241328,65.861638,0.991324,-0.513463
47,0.181628,0.531858,6.892402,54.008102,0.992965,-0.513691
54,0.171070,1.971115,7.168901,48.516582,0.998621,-0.513717
101,0.401026,0.014076,7.689483,49.849235,0.999282,-0.513761
118,0.143883,1.963752,6.357500,31.688469,0.999016,-0.513777
97,0.188695,0.304747,7.041766,51.069938,0.994610,-0.513808
69,0.296286,1.972459,5.713547,70.004185,0.991461,-0.513844


In [54]:
BO_scores.loc[-1] = [ 0.4699,1.9776, 6.7747,19.4144,0.7892, -0.52662 ]  # adding a row
BO_scores.index = BO_scores.index + 1 

In [ ]:
BO_scores = BO_scores.sort_values(by='score',ascending=False)

BO_scores.head()

### Re-train models

Now we have optimized parameters, let's decrease the size of learning rate and train the model for better results.

Firstly we'll use xgb.cv again to get optimal n_estimators, then we can use tuned n_esimator to finally train the model.

In [57]:
params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.01
params['max_depth'] = int(BO_scores.to_dict()['max_depth'][0])
params['min_child_weight'] = BO_scores.to_dict()['min_child_weight'][0]
params['colsample_bytree'] = BO_scores.to_dict()['colsample_bytree'][0]
params['subsample'] = BO_scores.to_dict()['subsample'][0]
params['gamma'] = BO_scores.to_dict()['gamma'][0]
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_x, label=train_y.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=10,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(500)])

best_iteration = len(cv_results)


Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 500 rounds.
Stopping. Best iteration:
[4665]	train-mlogloss:0.333136+0.0012015	test-mlogloss:0.521276+0.00964687



In [58]:
clf = xgb.XGBClassifier(n_estimators = best_iteration,
                              learning_rate=0.01,
                              max_depth=int(BO_scores.to_dict()['max_depth'][0]),
                              min_child_weight=BO_scores.to_dict()['min_child_weight'][0],
                              colsample_bytree=BO_scores.to_dict()['colsample_bytree'][0],
                              subsample=BO_scores.to_dict()['subsample'][0],
                              gamma=BO_scores.to_dict()['gamma'][0],
                              seed=1234,
                              nthread=-1)

clf.fit(train_x, train_y)

feature_importance = pd.DataFrame(sorted(zip(full_vars,clf.feature_importances_)
                          , key=lambda x: x[1], reverse = True),columns=['feature_name','importance']) 

print (feature_importance.query('importance>0'))


preds = clf.predict_proba(test_x)
sub_df = pd.DataFrame(preds,columns = ["low", "medium", "high"])
sub_df["listing_id"] = test_data.listing_id.values
sub_df.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\sub_xgb_tuned_v8.csv", index=False)

                                feature_name  importance
0                       hcc_building_id_high    0.031839
1                     hcc_building_id_medium    0.031707
2                          price_per_bedroom    0.030547
3                                      price    0.030518
4                        hcc_manager_id_high    0.030303
5                price_percentile_by_manager    0.030082
6                             price_per_room    0.028758
7                             dist_to_center    0.028652
8              price_percentile_by_disp_addr    0.027419
9                       manager_level_medium    0.026829
10                           price_per_photo    0.025932
11                              avg_word_len    0.025616
12       created_epoch_percentile_by_manager    0.025510
13                     hcc_manager_id_medium    0.025508
14                                  latitude    0.025462
15              price_percentile_by_building    0.025379
16                         mana

In [174]:
import lightgbm as lgb
lgb_train = lgb.Dataset(train_x, train_y)

def lgb_evaluate(max_bins,
                 num_leaves,
                 min_sum_hessian_in_leaf,
                 min_gain_to_split,
                 feature_fraction,
                 bagging_fraction,
                 bagging_freq
                 ):
    params = dict()
    params['objective'] = 'multiclass'
    params['num_class'] = 3
    params['learning_rate'] = 0.1
    params['max_bins'] = int(max_bins)   
    params['num_leaves'] = int(num_leaves)    
    params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
    params['min_gain_to_split'] = int(min_gain_to_split)    
    params['feature_fraction'] = feature_fraction
    params['bagging_fraction'] = bagging_fraction
    params['bagging_freq'] = int(bagging_freq)


    cv_results = lgb.cv(params,
                    lgb_train,
                    num_boost_round=100000,
                    nfold=5,
                    early_stopping_rounds=100,
                    metrics='multi_logloss',
                    verbose_eval=False
                   )

    return -pd.DataFrame(cv_results)['multi_logloss-mean'].min()


lgb_BO = BayesianOptimization(lgb_evaluate, 
                             {'max_bins': (127, 1023),
                              'num_leaves': (15, 512),
                              'min_sum_hessian_in_leaf': (1, 100),
                              'min_gain_to_split': (0,2),
                              'feature_fraction': (0.2, 0.8),
                              'bagging_fraction': (0.7, 1),
                              'bagging_freq': (1, 5)
                             }
                            )

lgb_BO.maximize(init_points=7, n_iter=150)

Initialization
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   bagging_fraction |   bagging_freq |   feature_fraction |   max_bins |   min_gain_to_split |   min_sum_hessian_in_leaf |   num_leaves | 
    1 | 00m46s |   -0.52299 |             0.9268 |         2.0500 |             0.6182 |  1004.7449 |              0.6965 |                   13.2418 |     111.0091 | 
    2 | 00m32s |   -0.52186 |             0.8552 |         1.5082 |             0.6259 |   467.5715 |              1.3656 |                   11.4053 |     319.3626 | 
    3 | 00m40s |   -0.52377 |             0.8099 |         2.0632 |             0.3571 |  1000.0635 |              0.7264 |                   72.6524 |     127.8515 | 
    4 | 00m28s |   -0.52192 |             0.7392 |         3.9515 |             0.6429 |   929.7169 |              1.5374 |                   75.2

In [175]:
## Show tuning results
lgb_BO_scores = pd.DataFrame(lgb_BO.res['all']['params'])
lgb_BO_scores['score'] = pd.DataFrame(lgb_BO.res['all']['values'])
lgb_BO_scores = lgb_BO_scores.sort_values(by='score',ascending=False)
lgb_BO_scores.head()

,bagging_fraction,bagging_freq,feature_fraction,max_bins,min_gain_to_split,min_sum_hessian_in_leaf,num_leaves,score
67,0.878972,4.960192,0.298949,166.653383,0.783663,31.274728,15.732732,-0.518042
23,0.905694,3.266224,0.364004,1018.838113,1.683158,4.315071,17.229199,-0.518704
0,0.867569,4.032703,0.266497,608.197952,1.102316,1.372453,20.086930,-0.518795
56,0.871087,1.187484,0.420960,633.178880,1.159392,2.050158,17.003101,-0.518920
24,0.815361,2.497831,0.746002,708.200014,1.859532,5.423963,15.849160,-0.518983


In [176]:
# LightGBM
clfs = []
lgb_clfs = []
for p in lgb_BO_scores.head(10).iterrows():
    clfs.append(lgb.LGBMClassifier(n_estimators = 10000,
                                    learning_rate=0.01,
                                    max_bin=int(p[1].to_dict()['max_bins']),
                                    num_leaves=int(p[1].to_dict()['num_leaves']),
                                    min_child_weight=int(p[1].to_dict()['min_sum_hessian_in_leaf']),
                                    colsample_bytree=p[1].to_dict()['feature_fraction'],
                                    subsample=p[1].to_dict()['bagging_fraction'],
                                    subsample_freq=int(p[1].to_dict()['bagging_freq']),
                                    min_split_gain=p[1].to_dict()['min_gain_to_split'],
                                    seed=1234,
                                    nthread=-1)
                )

# XGBoost    
xgb_clfs = []
for p in BO_scores.head(10).iterrows():
    clfs.append(xgb.XGBClassifier(n_estimators = 10000,
                                  learning_rate=0.01,
                                  max_depth=int(p[1].to_dict()['max_depth']),
                                  min_child_weight=int(p[1].to_dict()['min_child_weight']),
                                  colsample_bytree=p[1].to_dict()['colsample_bytree'],
                                  subsample=p[1].to_dict()['subsample'],
                                  gamma=p[1].to_dict()['gamma'],
                                  seed=1234,
                                  nthread=-1))


In [182]:
interest_levels = ['low', 'medium', 'high']

tau = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df):
    y = df[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print(a)

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print(a)

    return df_correct

In [61]:
sub_df2 = correct(sub_df)
sub_df2.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\sub_xgb_tuned_v8.1.csv", index=False)

[0.99025408244876045, 1.0199119751777115, 1.0307702227597324]
[0.9956691306468165, 1.0076912404379388, 1.0164188334778979]


In [177]:
def blend_model(clfs, train_x, train_y, test_x, num_class, blend_folds):
    num_class = 3
    blend_folds = 5

    skf = model_selection.StratifiedKFold(n_splits=blend_folds,random_state=1234)
    skf_ids = list(skf.split(train_x, train_y))


    train_blend_x = np.zeros((train_x.shape[0], len(clfs)*num_class))
    test_blend_x = np.zeros((test_x.shape[0], len(clfs)*num_class))
    blend_scores = np.zeros ((blend_folds,len(clfs)))

    print  ("Start blending.")
    for j, clf in enumerate(clfs):
        print ("Blending model",j+1, clf)
        test_blend_x_j = np.zeros((test_x.shape[0], num_class))
        for i, (train_ids, val_ids) in enumerate(skf_ids):
            print ("Model %d fold %d" %(j+1,i+1))
            train_x_fold = train_x[train_ids]
            train_y_fold = train_y[train_ids]
            val_x_fold = train_x[val_ids]
            val_y_fold = train_y[val_ids]
            # Set n_estimators to a large number for early_stopping
            clf.n_estimators = 10000000
            
            # Set evaluation metric
            if type(clf).__name__=='LGBMClassifier':
                metric = 'logloss' #LightGBM
            else:
                metric = 'mlogloss' #XGBoost            
            clf.fit(train_x_fold, train_y_fold,
                    eval_set=[(val_x_fold,val_y_fold)],
                    eval_metric=metric,
                    early_stopping_rounds=500,verbose=False)
            val_y_predict_fold = clf.predict_proba(val_x_fold)
            score = metrics.log_loss(val_y_fold,val_y_predict_fold)
            print ("LOGLOSS: ", score)
            print ("Best Iteration:", clf.best_iteration)
            blend_scores[i,j]=score
            train_blend_x[val_ids, j*num_class:j*num_class+num_class] = val_y_predict_fold
            test_blend_x_j = test_blend_x_j + clf.predict_proba(test_x)
        test_blend_x[:,j*num_class:j*num_class+num_class] = test_blend_x_j/blend_folds
        print ("Score for model %d is %f" % (j+1,np.mean(blend_scores[:,j])))
    return train_blend_x, test_blend_x, blend_scores

In [ ]:
def blend_model2(clfs, train_x, train_y, test_x, num_class, blend_folds):
    num_class = 3
    blend_folds = 5

    skf = model_selection.StratifiedKFold(n_splits=blend_folds,random_state=1234)
    skf_ids = list(skf.split(train_x, train_y))


    train_blend_x = np.zeros((train_x.shape[0], len(clfs)*num_class))
    test_blend_x = np.zeros((test_x.shape[0], len(clfs)*num_class))
    blend_scores = np.zeros ((blend_folds,len(clfs)))

    print  ("Start blending.")
    for j, clf in enumerate(clfs):
        print ("Blending model",j+1, clf)
        test_blend_x_j = np.zeros((test_x.shape[0], num_class))
        for i, (train_ids, val_ids) in enumerate(skf_ids):
            print ("Model %d fold %d" %(j+1,i+1))
            train_x_fold = train_x[train_ids]
            train_y_fold = train_y[train_ids]
            val_x_fold = train_x[val_ids]
            val_y_fold = train_y[val_ids]
            clf.fit(train_x_fold, train_y_fold,
                    eval_set=[(val_x_fold,val_y_fold)],
                    eval_metric=metric,
                    early_stopping_rounds=500,verbose=False)
            val_y_predict_fold = clf.predict_proba(val_x_fold)

In [72]:
xgb_clfs = []
for p in BO_scores.head(4).iterrows():
    xgb_clfs.append(xgb.XGBClassifier(n_estimators = 10000,
                                  learning_rate=0.01,
                                  max_depth=int(p[1].to_dict()['max_depth']),
                                  min_child_weight=int(p[1].to_dict()['min_child_weight']),
                                  colsample_bytree=p[1].to_dict()['colsample_bytree'],
                                  subsample=p[1].to_dict()['subsample'],
                                  gamma=p[1].to_dict()['gamma'],
                                  seed=1234,
                                  nthread=-1))

In [178]:
train_blend_x, test_blend_x, blend_scores_x = blend_model(clfs, 
                                                        train_x, 
                                                        train_y, 
                                                        test_x, 
                                                        num_class=3, 
                                                        blend_folds=4)

Start blending.
Blending model 1 LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.29894906310327574,
        drop_rate=0.1, is_unbalance=False, learning_rate=0.01, max_bin=166,
        max_depth=-1, max_drop=50, min_child_samples=10,
        min_child_weight=31, min_split_gain=0.78366256935552192,
        n_estimators=10000, nthread=-1, num_leaves=15, objective='binary',
        reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=1234,
        sigmoid=1.0, silent=True, skip_drop=0.5,
        subsample=0.87897197295970197, subsample_for_bin=50000,
        subsample_freq=4, uniform_drop=False, xgboost_dart_mode=False)
Model 1 fold 1
LOGLOSS:  0.540670778854
Best Iteration: 6225
Model 1 fold 2
LOGLOSS:  0.49972019773
Best Iteration: 7312
Model 1 fold 3
LOGLOSS:  0.500530066693
Best Iteration: 6125
Model 1 fold 4
LOGLOSS:  0.506867485999
Best Iteration: 5489
Model 1 fold 5
LOGLOSS:  0.532422423891
Best Iteration: 4289
Score for model 1 is 0.516042
Blending model 2 LGBMClassifier(boo

In [221]:
pd.DataFrame(train_blend_x).shape

(49352, 60)

In [179]:
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neural_network import MLPClassifier
def search_model(train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = model_selection.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring    = 'log_loss',
                                     verbose    = 10,
                                     n_jobs  = n_jobs,
                                     iid        = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

In [180]:
param_grid = {"hidden_layer_sizes":[10,50,100,200,300, 500,800]
              }
model = search_model(train_blend_x
                                         , train_y
                                         , MLPClassifier()
                                         , param_grid
                                         , n_jobs=-1
                                         , cv=5
                                         , refit=True)   

print ("best subsample:", model.best_params_)




Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:   53.3s remaining:   24.4s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:  1.2min remaining:   18.2s
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:  1.8min remaining:    9.9s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:  2.2min finished


Best score: -0.513
Best parameters set: {'hidden_layer_sizes': 100}
Scores: [mean: -0.51687, std: 0.01564, params: {'hidden_layer_sizes': 10}, mean: -0.51616, std: 0.01666, params: {'hidden_layer_sizes': 50}, mean: -0.51347, std: 0.01586, params: {'hidden_layer_sizes': 100}, mean: -0.51491, std: 0.01519, params: {'hidden_layer_sizes': 200}, mean: -0.51649, std: 0.01479, params: {'hidden_layer_sizes': 300}, mean: -0.51547, std: 0.01578, params: {'hidden_layer_sizes': 500}, mean: -0.51374, std: 0.01674, params: {'hidden_layer_sizes': 800}]
best subsample: {'hidden_layer_sizes': 100}


In [181]:
preds = model.predict_proba(test_blend_x)
sub_df = pd.DataFrame(preds,columns = ["low", "medium", "high"])
sub_df["listing_id"] = test_data.listing_id.values
sub_df.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions/sub_all_v2.csv", index=False)

In [183]:
sub_df2 = correct(sub_df)
sub_df2.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\all_corrected_v2.csv", index=False)

[0.98778984534963388, 1.0135358429239887, 1.0764807875146518]
[0.99342071698107903, 1.010199501846899, 1.0301138452209655]


In [229]:
pdtrain_blend_x

PandasError: DataFrame constructor not properly called!

In [222]:
train_blend_x = sparse.hstack([train_x, train_blend_x]).tocsr()
test_blend_x = sparse.hstack([test_x, test_blend_x]).tocsr()

In [227]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 120)
svd.fit_transform(train_blend_x)

ValueError: array must not contain infs or NaNs

In [223]:
params = dict()
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eta'] = 0.01
params['max_depth'] = 7
params['min_child_weight'] = 15
params['subsample'] = 1
params['colsample_bytree'] = 0.33
params['gamma'] = 0.35
params['seed']=1234

cv_results = xgb.cv(params, xgb.DMatrix(train_blend_x, label=train_y.reshape(train_x.shape[0],1)),
               num_boost_round=1000000, nfold=5,
       metrics={'mlogloss'},
       seed=1234,
       callbacks=[xgb.callback.early_stop(50)])

best_xgb_score = cv_results['test-mlogloss-mean'].min()
best_xgb_iteration = len(cv_results)

Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[745]	train-mlogloss:0.430923+0.00170042	test-mlogloss:0.513554+0.00637356



In [224]:
clf = xgb.XGBClassifier(learning_rate = 0.01
                  , n_estimators =best_xgb_iteration
                  , max_depth = 7
                  , min_child_weight = 15
                  , subsample = 1
                  , colsample_bytree = 0.33
                  , gamma = 0.35
                  , seed = 1234
                  , nthread = -1
                  )

clf.fit(train_blend_x, train_y)

preds = clf.predict_proba(test_blend_x)
sub_df3 = pd.DataFrame(preds,columns = ["low", "medium", "high"])
sub_df3["listing_id"] = test_data.listing_id.values
sub_df3.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\final.csv", index=False)

In [225]:
sub_df4 = correct(sub_df3)
sub_df4.to_csv(r"C:\Users\Desktop\Kaggle Comptetion\Current Competition\Predictions\final_v2.csv", index=False)

[1.0020531527463894, 1.0092896711547257, 0.95612504248874741]
[1.0018611752049671, 0.99838347046353215, 0.98846793058918203]
